# Lightning Metrics Chart Demo

The following document contains a demos of chart that is possible make from the metrics available on the [lightning-metrics-rfc](https://github.com/OpenLNMetrics/lightning-metrics-rfc)

## Table of Content

- Introduction: An introduction on lightning-metrics-rfc;
- Description of The Metrics: A summary of the metrics that are implemented in this demo;
- Chart from Metrics: A implementation of the chart shows described in the previous section;
- Contributions: How contribute in the demo with other nodes data;
- Donations: Contribute if you like the idea to have a Open Source Metrics of the Lightning Network.

## Introduction

TODO

## Description of The Metrics

### Metrics One

TODO

In [1]:
from src import ChartMaker

maker = ChartMaker(base_dir="resources")

maker.make_payments_analysis_by_node("03e2408a49f07d2f4083a47344138ef89e7617e63919202c92aa8d49b574a560ae").render_notebook()

In [2]:
maker.make_node_status("03e2408a49f07d2f4083a47344138ef89e7617e63919202c92aa8d49b574a560ae").render_notebook()

## Chart from Metrics

### Metrics One

TODO

## Contributions

TODO

## Donations

TODO